In [99]:
from langchain.document_loaders import PyPDFLoader
from langchain_core.documents.base import Document

pfile = "/home/gs/Downloads/Alibaba Group Announces December Quarter 2023 Results.pdf"
# pfile = "/home/gs/Downloads/INVOICE _ GAO Sheng.pdf"
pdf_loader = PyPDFLoader(pfile)

# text_splitter = RecursiveCharacterTextSplitter(#separators=["\n"], 
#                                                chunk_size = 256, chunk_overlap=8, keep_separator = True)
pages = pdf_loader.load() #_and_split(text_splitter)

# vector_index = Chroma.from_documents(pages, llm_embed_model).as_retriever(
#     # search_type="similarity_score_threshold",
#     search_kwargs={'k': 20}
# )
# pages[3]
# Document(page_content="", metadata={"a":1})
# para_meta

In [129]:
from pypdf import PdfReader
from copy import deepcopy

reader = PdfReader(pfile)
pages2 = list(map(lambda x: x.extract_text(extraction_mode="layout"), reader.pages))

def paragraph_process(doc_pages, source = None, company = None, year = None):
    def single_page(i_text, page_id, is_lower = True):
        i_text = i_text.lower() if is_lower() else i_text
        
        nn = len(i_text)
        pg_stack = [] #paragraph list
        ss = ""
        k = 0
        while k < nn:
            if i_text[k] == "\n":
                kp = k + 1
                rt_ct = 0
                while kp < nn:
                    if i_text[kp] == "\n":
                        rt_ct += 1
                        kp += 1
                    elif i_text[kp] == " ":
                        kp += 1
                        continue
                    else:
                        break
                if rt_ct >= 1:
                    #paragraph
                    # ss += "\n"
                    pg_stack.append(ss)
                    ss = ""
                    k = kp
                else:
                    ss += " "
                    k += 1
            elif i_text[k] == " ":
                #space
                kp = k + 1
                rt_ct = 0
                while kp < nn:
                    if i_text[kp] == " ":
                        rt_ct += 1
                        kp += 1
                    else:
                        break
                ss += " "
                k = kp            
            else:
                ss += i_text[k]
                k += 1
        # pg = list(map(lambda x: x.strip(), filter(lambda x: len(x) >= 1, ss.split("\n"))))
        try:
            pg_id = int(pg_stack[-1])
            pg_stack = pg_stack[:-1] if pg_id - 1 == page_id else pg_stack
        except:
            try:
                pg_id = int(pg_stack[0])
                pg_stack = pg_stack[1:] if pg_id - 1 == page_id else pg_stack
            except:
                pg_id = None
        
        # print(f"{page_id}, {pg_id}, {pg_stack}")
        # print(f"** {ss}, {k}")
        return pg_stack

    #process page-by-page
    para_docs = []
    para_doc_pid = []
    for pid, pcontent in enumerate(doc_pages):
        ss = single_page(pcontent, pid)
        para_docs.extend(ss)
        para_doc_pid.extend([{"page": pid, "source": source, "company": company, "year": 2023}] * len(ss))

    return para_docs, para_doc_pid

def chunk_create(content_lst: list[str], metadata_lst: list[{}] = None, chunk_size: int = 256):
    k = 0
    chunk_stack = []
    chunk_meta_stack = []
    chunk_str = ""
    nn = len(content_lst)
    metadata_lst = metadata_lst if metadata_lst else [{}] * nn
    doc_ct = 0
    while k < nn:
        if len(chunk_str.split(" ")) >= chunk_size:
            chunk_meta = metadata_lst[k - 1]
            chunk_meta["docid"] = deepcopy(doc_ct)              
            tmpdoc = Document(page_content = chunk_str.strip(), metadata=chunk_meta)
            chunk_stack.append(tmpdoc)
            chunk_str = content_lst[k]
            doc_ct += 1
        else:
            tmpstr = chunk_str + " " + content_lst[k]
            if len(tmpstr.split(" ")) > chunk_size and len(content_lst[k].split(" ")) >= chunk_size//2:
                chunk_meta = metadata_lst[k - 1]
                chunk_meta["docid"] = deepcopy(doc_ct)                
                tmpdoc = Document(page_content = chunk_str.strip(), metadata=chunk_meta)
                chunk_stack.append(tmpdoc)
                # chunk_stack.append(chunk_str.strip())
                chunk_str = content_lst[k]
                doc_ct += 1
            else:
                chunk_str = tmpstr
        # print(f"** {k}, {chunk_str}, {content_lst[k]}")
        k += 1
        
    if len(chunk_str.split(" ")) > 0 and len(chunk_str.split(" ")) < chunk_size//2:
        if len(chunk_stack): 
            tmpstr = chunk_stack[-1].page_content + " " + chunk_str
            chunk_meta = metadata_lst[nn - 1]
            chunk_meta["docid"] = deepcopy(doc_ct) 
            chunk_stack[-1] = Document(page_content = tmpstr.strip(), metadata=chunk_meta)
        else:
            chunk_meta = metadata_lst[nn - 1]
            chunk_meta["docid"] = deepcopy(doc_ct) + 1
            tmpdoc = Document(page_content = tmpstr.strip(), metadata=chunk_meta)
            chunk_stack.append(tmpdoc)
            # chunk_stack.append(chunk_str.strip())
    else:
        chunk_meta = metadata_lst[nn - 1]
        chunk_meta["docid"] = deepcopy(doc_ct) + 1
        tmpdoc = Document(page_content = tmpstr.strip(), metadata=chunk_meta)
        chunk_stack.append(tmpdoc)
        # chunk_stack.append(chunk_str.strip())

    # print(chunk_stack)
    return chunk_stack
            
    #
    

# idd = 0
# para_docs = []
# chunk_docs = []
# para_doc_pid = []
# for pid, pcontent in enumerate(pages2):
#     ss = paragraph_process(pcontent, reader.get_page_number(reader.pages[pid]))
#     para_docs.extend(ss)
#     para_doc_pid.extend([pid] * len(ss))

para_docs, para_meta = paragraph_process(pages2, source = pfile, company = "alibaba", year = 2023)
chunk_docs = chunk_create(para_docs, para_meta, chunk_size = 256)
# print("\n***".join(ss.split("\n")))

# print("\n***".join(para_docs))
# print("\n***".join(chunk_docs))
# print(len(pages2[idd]))
# pages2[idd]
# print(pages2[-1])#.replace("  ", ""))#.replace("\n", " "))
# for v in reader.stream():
#     print(v)


TypeError: 'bool' object is not callable

In [118]:
# chunk_docs

In [127]:
import pypdf
import os
from llama_cpp import Llama
import pydantic
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyPDFLoader
from tempfile import NamedTemporaryFile
from langchain.vectorstores import Chroma, FAISS
import json
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

model_name = "/home/gs/hf_home/models/models--google--gemma-2b-it/gemma-2b-it.gguf"
model_name_embed = "/home/gs/hf_home/models/models--google--gemma-2b/gemma-2b.gguf"

#define consistent parametes
# n_batch >= chunk-size
chunk_size = 512

llm_embed_model = LlamaCppEmbeddings(
    model_path = model_name_embed, 
    n_gpu_layers = -1, 
    n_ctx = 512 * 4, 
    n_batch = chunk_size, 
    verbose=False
)
llm_chat_model = LlamaCpp(
        model_path=model_name,
        n_gpu_layers=-1,
        n_batch = chunk_size,
        # callback_manager=callback_manager,
        n_ctx=1024*4, # Uncomment to increase the context window
        # temperature=0.75,
        # f16_kv=True,
        verbose=False,  # Verbose is required to pass to the callback manager
)


In [97]:
# chunk_docs2 = list(map(lambda x: x.lower(), chunk_docs))
# para_docs

In [120]:
db = FAISS.from_texts(chunk_docs2, llm_embed_model).as_retriever()
doc = db.get_relevant_documents(
    """
    chief executive officer
    """.lower()
)
doc


In [130]:
from langchain_community.retrievers import BM25Retriever, TFIDFRetriever

db2 = TFIDFRetriever.from_texts(chunk_docs2, k = 4)
doc2 = db2.get_relevant_documents(
    "who is chief exective officer?"
   )
doc2


[Document(page_content='  alibaba group announces december quarter 2023 results hangzhou, china, february 7 , 2024 – alibaba group holding limited (nyse: baba and hkex: 9988 (hkd counter) and 89988 (rmb counter), “alibaba” or “alibaba group”) today announced its financial results for the quarter ended december 31, 2023. “we delivered a solid quarter as we are executing our focused strategies across the organization. our top priority is to reignite the growth of our core businesses, e -commerce and cloud computing. we will step up investment to improve user s ’ core experiences to drive growth in taobao and tmall group and strengthen market leadership in the coming year. we will also focus our resources on developing public cloud products and sustaining the strong growth momentum in international commerce business ,” said eddie wu, chief executive officer of alibaba group. “alibaba group delivered a healthy quarter with revenue growth of 5% year -over -year. we in creased our investment

In [57]:
chunk_docs2[0]

'  alibaba group announces december quarter 2023 results hangzhou, china, february 7 , 2024 – alibaba group holding limited (nyse: baba and hkex: 9988 (hkd counter) and 89988 (rmb counter), “alibaba” or “alibaba group”) today announced its financial results for the quarter ended december 31, 2023. “we delivered a solid quarter as we are executing our focused strategies across the organization. our top priority is to reignite the growth of our core businesses, e -commerce and cloud computing. we will step up investment to improve user s ’ core experiences to drive growth in taobao and tmall group and strengthen market leadership in the coming year. we will also focus our resources on developing public cloud products and sustaining the strong growth momentum in international commerce business ,” said eddie wu, chief executive officer of alibaba group. “alibaba group delivered a healthy quarter with revenue growth of 5% year -over -year. we in creased our investment in strategic prioritie

In [139]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.document_loaders import TextLoader

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": db2, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm_chat_model | output_parser

query = "what is the number of revenue growth rate?"

chain.invoke(query)

'Answer: The number of revenue growth rate is not explicitly mentioned in the context, so I cannot answer this question from the provided context.'

In [142]:
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
stuff_chain = load_qa_chain(llm_chat_model, chain_type="stuff", prompt=prompt)
question = "what is revenue growth rate?" #"how many revenue achieved in 2023" # #"how many revenue achieved?" ##"how many employees by end of 2023?" #
context_str = db2.get_relevant_documents(question)#, k=50, fetch_k=100)
stuff_answer = stuff_chain(
    {"input_documents": context_str, "question": question}, return_only_outputs=True
)
print(stuff_answer["output_text"])

The revenue growth rate is calculated by dividing the difference between current year revenue and previous year revenue by the previous year revenue. 

Therefore, in this case, the revenue growth rate for taobao and tmall group in the quarter ended December 31, 2023, would be calculated as follows: 

(rmb129,070 million - rmb122,736 million) / rmb122,736 million = 2%


In [141]:
context_str

[Document(page_content='(2) all others include sun art, freshippo, alibaba health, lingxi games, intime, intelligent information platform (which mainly  consists of ucweb and quark businesses), fliggy, dingtalk (previously reported under cloud intelligence group segment)  and other bus inesses. (3) unallocated primarily relate s to certain costs incurred by corporate functions and other miscellaneous items that are not  allocated to individual segments. (4) for a more intuitive presentation, widening of loss in yoy% is shown in terms of negative growth rate, and narrowing of loss  in yoy% is shown in terms of positive growth rate. taobao and tmall group (i) s egment revenue • china commerce retail business revenue from our china commerce retail business in the quarter ended december 31, 2023 was  rmb123,762 million (us$17,432 million), an increase of 1% compared to rmb122,736 million in  the same quarter of 2022. customer management revenue remained stable year -over -year, primarily d